# 《Reinforcement Learning, Second Edition 2018》笔记

![](https://github.com/applenob/rl_learn/raw/master/res/rl_book.jpg)

## 目录

- [0.书本介绍](#0.书本介绍)
- [1. Introduction](#1.-Introduction)
- [2. Multi-armed Bandits](#2.-Multi-armed-Bandits)
- [3. Finite Markov DecisionProcesses](#3.-Finite-Markov-DecisionProcesses)
- [4. Dynamic Programming](#4.-Dynamic-Programming)
- [5. Monte Carlo Methods](#5.-Monte-Carlo-Methods)
- [6. Temporal-Difference Learning](#6.-Temporal-Difference-Learning)
- [7. n-step Bootstrapping](#7.-n-step-Bootstrapping)
- [8. Planning and Learning with Tabular Methods](#8.-Planning-and-Learning-with-Tabular-Methods)
- [9. On-policy Prediction with Approximation](#9.-On-policy-Prediction-with-Approximation)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)



## 0.书本介绍

[Reinforcement Learning](https://book.douban.com/subject/2866455/)

作者: Richard S. Sutton / Andrew G. Barto 

几乎是最权威的强化学习教材。

### 重要的符号表示
- $q_∗(a)$：动作a对应的真实价值（价值期望），true value (expected reward) of action a
- $Q_t(a)$：动作a在t时刻的估计价值，estimate at time t of q∗(a)
- $N_t(a)$：在t时刻之前，动作a已经被选择的次数，number of times action a has been selected up prior to time t
- $H_t(a)$：在t时刻选择动作a的偏好程度，learned preference for selecting action a at time t
- $π_t(a)$：在t时刻选择动作a的概率，probability of selecting action a at time t
- $R_t$：给定策略πt，在t时刻的期望奖励，estimate at time t of the expected reward given πt


## 1. Introduction

### 什么是强化学习
强化学习是“学习该如何去做”（learning what to do），即学习如何从**某个状态映射到某个行为，来最大化某个数值的奖励信号**。

### 强化学习的特征
强化学习两个最重要的特征：
- **试错（trial-and-error search ）**：agent需要不断尝试，通过reward的反馈学习策略。
- **延迟奖励（delayed reward） **：某个时刻的action可能会对后面时刻的reward有影响（深远影响）。

### Exploit vs Explore
**exploit** 代表利用已有的信息去获得最大的奖励，**explore** 代表去探索一些没有尝试过的行为，去获得可能的更大的奖励。

### 强化学习的几个要素
- **policy**: 状态到行为的映射，定义agent在某个状态下应该如何采取行为，state $\rightarrow$ action。
- **reward function**: 在某个状态下，agent能够收到的**即时反馈**。
- **value function**: 衡量在某个状态下，能够获得的**长期反馈**。
- **modle (of the environment，可选的)**: 模型用来模拟环境的行为，**给定一个状态和动作，模型可以预测下一个状态和奖励**。
   
### RL vs Evolutionary Methods
- Evolutionary Methods（遗传算法，具体可以回顾[之前的博客](https://applenob.github.io/ga.html)），直接在policy空间中搜索，并计算最后的得分。通过一代代的进化来找寻最优policy。
- 遗传算法忽略了policy实际上是state到action的映射，它不关注agent和环境的互动，只看最终结果。

### 局限性
强化学习非常依赖**状态（state）**的概念。state既是策略函数和价值函数的输入，又是环境模型（model）的输入和输出。

### Tic-Tac-Toe（井字棋）
- ![](https://github.com/applenob/rl_learn/raw/master/res/ttt.png)
- 一个简单的应用强化学习的例子。
- 定义policy：任何一种局面下，该如何落子。
- 遗传算法解法：试很多种policy，找到最终胜利的几种，然后结合，更新。
- 强化学习解法：
    - 1.建立一张表格，state_num × 1，代表每个state下，获胜的概率，这个表格就是所谓的**value function**，即状态到价值的映射。
    - 2.跟对手下很多局。每次落子的时候，依据是在某个state下，选择所有可能的后继state中，获胜概率最大的（value最大的）。这种方法即贪婪法（Exploit）。偶尔我们也随机选择一些其他的state（Explore）。
    - 3.“back up”后继state的v到当前state上。$V(s)\leftarrow V(s)+\alpha[V(s')-V(s)]$，这就是所谓的**差分学习（temporal-difference learning）**，这么叫是因为$V(s')-V(s)$是两个时间点上的两次估计的差。

## 2. Multi-armed Bandits

### k-armed Bandit Problem，K臂老虎机
- 问题描述：重复地面临有k中选择的情况，每一次选择之后，都会收到一个reward。这个reward服从一个跟你的选择相关的分布。你的目标是，在选择t次后，找到最大的期望reward。
- 为什么叫K臂老虎机：有K个单臂老虎机，每个时间节点，你可以选择K个中任意一个老虎机选择按下它的臂，然后获得一个奖励。目标自然是多次选择之后的累计奖励最大。
- 形式化定义：假设时刻$t$的动作是$A_t$，reward是$R_t$，定义价值函数：$q_*(a) = E[R_t|A_t=a]$。
- 特点：每个时刻下，$q_*(a)$是固定的，stationary。
- 如果我们知道每个动作a对应的价值q，那么这个问题对我们来说就已经解决了，即，我们选择q最大的那个a。
- 然而，我们并不知道实际上的q是什么，只能估计$Q_t(a)\approx q_*(a)$。
- 在每个时间节点上，如果总是选择对应的估计Q最大的a，这被称为**greedy**。
- **$\epsilon$-greedy**：*：$1-\epsilon$的概率选择Q最大的a，*$\epsilon$的概率选择其他a。
- 估计$Q_t(a) = \frac{在时刻t之前，所有采用动作a获得的奖励之和}{在时刻t之前，采用动作a的次数} = \frac{\sum^{t-1}_{i=1}R_i\cdot\mathbf{1}_{A_i=a}}{\sum^{t-1}_{i=1}\mathbf{1}_{A_i=a}}$，被称为sample-average。

### 10-armed Testbed
- 设置一个10臂老虎机作模拟。
- 真实的价值函数$q_*(a)$服从标准高斯分布（均值是0，方差是1）。
- t时刻的奖励$R_t$服从高斯分布，均值是$q_*(A_t)$，方差是1。
- 可以规定每次实验做1000次选择，称为一个run。
- 一共跑了2000个独立的run。
- 实验结果（**$\epsilon$-greedy**的优越性）：
- ![](https://github.com/applenob/rl_learn/raw/master/res/average_reward.png)

### 增量式的估计
- 上面的sample-average即：$Q_n = \frac{R_1+R_2+...+R_{n-1}}{n-1}$
- 改写成增量式的形式：$Q_{n+1} = Q_n + \frac{1}{n}[R_n-Q_n]$
- 即：新估计 = 老估计 + 步长 × [奖励 - 老估计]
- ![](https://github.com/applenob/rl_learn/raw/master/res/bandit_algo.png)

### Optimistic Initial Values
- 设置初始的$Q_1(a)$为一些较大的值。
- 这样会鼓励explore。
- 对于nonstationary的问题不适用。
- ![](https://github.com/applenob/rl_learn/raw/master/res/optimistic.png)

### Upper-Confidence-Bound Action Selection（UCB）
- $A_t = \underset{a}{argmax}[Q_t(a)+c\sqrt{\frac{log\;t}{N_t(a)}}]$
- 其中，$N_t(a)$代表动作a在t之前出现的次数，根号项衡量动作a的不确定性，如果某个动作已经使用了很多次，则倾向使用使用次数少的，这样达到explore的效果。
- ![](https://github.com/applenob/rl_learn/raw/master/res/ucb.png)

### Gradient Bandit Algorithms
- 使用一个数值表示对某个动作的**偏好**：$H_t(a)$
- $Pr\{A_t=a\}=\frac{e^{H_t(a)}}{\sum_{b=1}^k{e^{H_t(b)}}}=\pi_t(a)$
- 更新规则（不必细究，学到policy gradient自然明白）：
    - $H_{t+1}(A_t) = H_t(A_t) + \alpha(R_t-\overline R_t)(1-\pi _t(A_t))$
    - $\forall a \neq A_t$
- ![](https://github.com/applenob/rl_learn/raw/master/res/gradient_bandit.png)

### 各种方法对比
- ![](https://github.com/applenob/rl_learn/raw/master/res/bandit_methods.png)

## 3. Finite Markov DecisionProcesses

### Agent和Environment的交互
- 学习者和决策者称为agent。
- agent交互的对象，外部环境，称为Environment。
- 在时刻t，agent的所处的环境用状态：$S_t \in S$表示，$S$是可能的状态集。假设agent采用了动作$A_t\in A(S_t)$，$A(S_t)$代表在状态$S_t$下可能的动作集。
- 到了下一个时刻t+1，agent收到了一个奖励：$R_{t+1} \in R$，并且发现自己处在一个新的state中：$S_{t+1}$。
- ![](https://github.com/applenob/rl_learn/raw/master/res/agent_env.png)

### 什么是有限
有限MDP中的有限意味着：状态空间S、动作空间A和奖励空间R都是离散且有限的。

### 目标和奖励的区别
- 每个时间节点上，agent都会收到一个奖励的数值：$R_t$。
- 但是，agent的目标应该是：所有时间结点上的奖励的和的最大化。
- 即：$G_t = R_{t+1} +  R_{t+2} + ... + R_T$

### 什么是Episode
一系列的agent和environment交互序列。每个episode之间相互不影响，且都是有一个相同的终止状态（terminate state）。

### Continuing Tasks
不同于Episode类的任务，这类任务没有终止状态，也就是说$T = \infty$。因此，如果考虑这类任务，那么$G_t$将是无穷大，于是引入discounting。

### Episode和Continuing Tasks的统一规范
在Episode的尾部加入吸收状态（absorbing state），在此状态下，奖励永远是0，且下一个状态永远是当前状态。

这样收益可以统一使用下面的Expected Discounted Return表示。

### Expected Discounted Return
- $G_t = R_{t+1} +  \gamma R_{t+2} + ... = \sum_{k=0}^{\infty} \gamma ^k R_{t+k+1}$
- $\gamma$是参数，且$0\leq \gamma \leq 1$，被称为**discount rate**。
- 含义：一个奖励，如果是在k个时间节点以后收到，那么对于当前来说，它的价值是即时奖励的$\gamma^{k-1}$倍。
- 从$G_t$的定义，很容易获得递推式：$G_t = R_{t+1} + \gamma G_{t+1}$

### 马尔科夫性质（Markov property）
- 核心思想：**当前state继承了所有的环境历史信息。**
- $Pr\{S_{t+1}=s', R_{t+1}=r|S_0,A_0,R_1,...,S_{t-1},A_{t_1},R_t,S_t,A_t\} = Pr\{S_{t+1}=s', R_{t+1}=r|S_t,A_t\}$
- 即便state是非马尔科夫的，我们也希望近似到马尔科夫。

### Markov Dicision Processes（MDP）
- 满足马尔科夫性质的强化学习任务称为MDP。
- 如果state和action空间有限，则称为有限MDP（涵盖了现代强化学习90%的问题）。
- 用$p(s',r|s,a)$表示$Pr\{S_{t+1}={s}', R_{t+1}=r|S_t,A_t\}$，这个条件概率是描绘了整个MDP的动态（Dynamics）。
- state-action期望奖励：$r(s,a) = E[R_{t+1}|S_t=s,A_t=a]=\sum_{r\in R}r\sum_{s'\in S}p(s', r|s,a)$
- 状态转移概率：$p(s'|s,a) = Pr\{S_{t+1}=s'|S_t=s, A_t=a\}=\sum_{r\in R}p(s', r| s, a)$
- state-action-nextstate期望奖励：$r(s, a, s') = E[R_{t+1}|S_t=s,A_t=a, S_{t+1}=s'] = \sum_{r\in R}r\frac{p(s',r|s,a)}{p(s'|s,a)}$

### 价值函数
- 关于策略$\pi$的state-value函数：$v_{\pi}(s) = E_{\pi}[G_t|S_t=s]=E_{\pi}[\sum_{k=0}^{\infty}\gamma^kR_{t+k+1}|S_t=s]$。
- 即，在使用策略$\pi$的前提下，衡量处于某个state有多好。
- 关于策略$\pi$的action-value函数：$q_{\pi}(a,s) = E_{\pi}[G_t|S_t=s,A_t=a] = E_{\pi}[\sum_{k=0}^{\infty}\gamma^kR_{t+k+1}|S_t=s,A_t=a]$。
- 即，在使用策略$\pi$的前提下，衡量处于某个state下，执行某个action有多好。

### Bellman Euqation
- **Bellman Expectation Euqation for $v_{\pi}$**：$v_{\pi}(s) = \sum_a\pi(a|s)\sum_{s',r}p(s',r|s,a)[r+\gamma v_{\pi}(s')]\;\;\forall s \in S$
- 理解：
    - 1.方括号中是根据后继状态的价值重新估计的价值函数，再在动作空间、后继状态空间和动作空间用相应的概率做加权求和。
    - 2.**表达的是某个状态的价值和其后继状态的价值之间的关系。**
- **backup**：是强化学习方法的核心，以时序意义上的回退，用下一个时刻的值去评估当前时刻的值。
- ![](https://github.com/applenob/rl_learn/raw/master/res/backup.png)

### 最优化价值函数
- $v_*(s) = \underset{\pi}{max}v_{\pi}(s)$
- $q_*(s,a) = \underset{\pi}{max}q_{\pi}(s,a)$
- **Bellman Optimality Euqation for $v_*(s)$**：$v_*(s)=\underset{a\in A(s)}{max}\sum_{s',r}p(s',r|s,a)[r+\gamma v_*(s')]$
- **Bellman Optimality Euqation for $q_*(s,a)$**：$q_*(s,a)=\sum_{s',r}p(s',r|s,a)[r+\gamma \underset{a'}{max}q_*(s', a')]$
- ![](https://github.com/applenob/rl_learn/raw/master/res/backup_opt.png)


## 4. Dynamic Programming

**DP方法简介**：
- 由于其大量的计算损耗，已经不实用，但理论上非常重要。
- 假设解决的问题是有限的MDP，即给定动作a，状态s，和奖励r，可以通过$p(s',r|s,a)$描述动态变化。

**Policy Evaludation**：
- 计算某个policy对应的价值函数。
- 更新方法：使用上一章讲的**Bellman Expectation Euqation for $v_{\pi}$**：$v_{\pi}(s) = \sum_a\pi(a|s)\sum_{s',r}p(s',r|s,a)[r+\gamma v_{\pi}(s')]\;\;\forall s \in S$
- ![](https://github.com/applenob/rl_learn/raw/master/res/iterative_policy_evaluation.png)

**Policy Improvement**：
- policy improvement theorem：对所有的$s \in S$，有$q_{\pi}(s, \pi'(s)) \geq v_{\pi}(s)$，则$v_{\pi'}\geq v_{\pi}(s)$，即策略$\pi'$优于策略$\pi$。
- greedy policy：$\pi'(s)=\underset{a}{argmax}q_{\pi}(s,a)=\underset{a}{argmax}\sum_{s', r}p(s', r|s,a)[r+\gamma v_{\pi}(s')]$

**Policy Iteration**：
- ![](https://github.com/applenob/rl_learn/raw/master/res/policy_iteration_2.png)
- 其中，E代表evaluation，I代表improvement。
- ![](https://github.com/applenob/rl_learn/raw/master/res/policy_iteration.png)

**Value Iteration**：
- $v_{k+1}(s)=\underset{a}{max} E[R_{t+1}+\gamma v_k(S_{t+1})|S_t=s, A_t=a]\\=\underset{a}{max}\sum_{s',r}p(s',r|s,a)[r+\gamma v_k(s')]$，实际上就是**Bellman Optimality Euqation for $v_*(s)$**。
- ![](https://github.com/applenob/rl_learn/raw/master/res/value_iteration.png)

## 5. Monte Carlo Methods

****：
![](https://github.com/applenob/rl_learn/raw/master/res/fv_mc_est.png)
$V(s)\leftarrow average(Return(s))$
![](https://github.com/applenob/rl_learn/raw/master/res/fv_mc_control.png)
![](https://github.com/applenob/rl_learn/raw/master/res/off_pol_mc_control.png)
****：

****：

**Off-Policy的概念**：
- 之前介绍的算法，在做实验时（生成episode），都是遵照最终的目标policy去选择动作。但是我们知道，一开始这个policy是很差的，于是考虑在生成episode的时候，直接使用一种较优的policy去指导它。这种方法即为**Off-Policy**，采用的policy，有别于最终的目标policy，`target policy`，称为`behavior policy`。

****：

****：

****：

****：

****：

****：

## 6. Temporal-Difference Learning


****：
![](https://github.com/applenob/rl_learn/raw/master/res/td0_est.png)

$V(S)\leftarrow V(S)+\alpha[R+\gamma V(S')-V(S)]$
![](https://github.com/applenob/rl_learn/raw/master/res/sarsa_2.png)
![](https://github.com/applenob/rl_learn/raw/master/res/sarsa_est.png)
$Q(S,A)\leftarrow Q(S,A)+\alpha[R+\gamma Q(S',A')-Q(S,A)]$
![](https://github.com/applenob/rl_learn/raw/master/res/q_learn_backup.png)

![](https://github.com/applenob/rl_learn/raw/master/res/q_learn.png)


****：价值函数更新：$Q(S,A)\leftarrow Q(S,A)+\alpha[R+\gamma Q(S',A')-Q(S, A)]$

****：

****：

****：

****：

****：

****：

****：

In [1]:
print "done"

done
